![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

## Entraînement du modèle dans un Notebook (QuantBook)

In [1]:
# ======================================
# NOTEBOOK QUANTCONNECT - ML EXAMPLE
# ======================================
from AlgorithmImports import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ta.trend import SMAIndicator
from ta.momentum import RSIIndicator

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# 1) INIT DU QuantBook
qb = QuantBook()

# 2) CHARGEMENT DES DONNÉES
symbol = qb.AddCrypto("BTCUSDT", Resolution.Daily, Market.Binance).Symbol
start_date = datetime(2022,1,1)
end_date   = datetime(2023,12,31)
history = qb.History(symbol, start_date, end_date, Resolution.Daily)
print(f"History bars: {len(history)}")

# 3) DATAFRAME PLAT
df = (history.reset_index()
      .pivot(index='time', columns='symbol', values='close')
      .rename(columns={symbol:'Close'}))
df.sort_index(inplace=True)

# 4) INDICATEURS
df["SMA20"]       = SMAIndicator(df["Close"], 20, fillna=True).sma_indicator()
df["RSI"]         = RSIIndicator(df["Close"], 14, fillna=True).rsi()
df["DailyReturn"] = df["Close"].pct_change()

df.dropna(inplace=True)

# 5) TARGET : Prochaine clôture monte ou pas
df["Target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)
df.dropna(inplace=True)

# 6) FEATURES
features = ["SMA20", "RSI", "DailyReturn"]
X = df[features].values
y = df["Target"].values

# Train/test split temporel
split_idx = int(len(X)*0.8)
X_train, y_train = X[:split_idx], y[:split_idx]
X_test,  y_test  = X[split_idx:], y[split_idx:]

# 7) ENTRAÎNEMENT
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
clf.fit(X_train, y_train)

# 8) ÉVALUATION
y_pred = clf.predict(X_test)
acc    = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")
print(classification_report(y_test, y_pred))

# 9) SAUVEGARDE DANS L'OBJECT STORE
model_key = "myCryptoMlModel.pkl"  # Clé d'enregistrement
path      = qb.ObjectStore.GetFilePath(model_key)
joblib.dump(clf, path)
qb.ObjectStore.Save(model_key)
print(f"Modèle sauvegardé en ObjectStore : {model_key}")
